In [3]:
import streamlit as st

import geemap, ee
import geopandas as gpd
from shapely.geometry import Polygon

st.set_page_config(layout="wide")
@st.cache_data
def ee_authenticate(token_name="EARTHENGINE_TOKEN"):
    geemap.ee_initialize(token_name=token_name)

def getNDVI(image):    
    # Normalized difference vegetation index (NDVI)
    ndvi = image.normalizedDifference(['B8','B4']).rename("NDVI")
    image = image.addBands(ndvi)

    return(image)

def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())


st.title("NDVI Map Canterbury Region")
ee_authenticate(token_name="EARTHENGINE_TOKEN")

aoi = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq('ADM1_NAME','Canterbury')).geometry()

NDVI_data = ee.ImageCollection('COPERNICUS/S2_SR').filterDate("2023-01-01","2023-01-31").filterBounds(aoi) \
    .map(getNDVI).map(addDate).median()

color = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']
pallete = {"min":0, "max":1, 'palette':color}
# initialize our map
map1 = geemap.Map()
map1.centerObject(aoi, 7)
map1.addLayer(NDVI_data.clip(aoi).select('NDVI'), pallete, "NDVI-Canterbury")

map1.addLayerControl()
map1

# map1.to_streamlit(height=700)


2023-05-29 15:02:58.756 No runtime found, using MemoryCacheStorageManager


Map(center=[-43.64309971284063, 171.44533925780135], controls=(WidgetControl(options=['position', 'transparent…

In [9]:
coordcan = aoi.coordinates().getInfo()
coordcan

[[[[172.6841908306292, -43.629677406208025],
   [172.6898494544276, -43.632914691038515],
   [172.69900395245472, -43.632272606563966],
   [172.7006092083493, -43.62998506235086],
   [172.69936962898902, -43.62791605242774],
   [172.69274786040626, -43.62582473703023],
   [172.6902463163535, -43.62329196365133],
   [172.6841908306292, -43.629677406208025]]],
 [[[169.54440400228475, -44.425729491274026],
   [169.5559976303391, -44.44505967966168],
   [169.6054981716861, -44.44781989489019],
   [169.61869705124118, -44.461629697441744],
   [169.61049676118338, -44.497518764236354],
   [169.60229655599105, -44.533407858024965],
   [169.6230937797445, -44.54505058730139],
   [169.65519925479393, -44.53892822742271],
   [169.68789788836318, -44.51447900140205],
   [169.71949955186736, -44.520030541006456],
   [169.73114887724648, -44.54227035263125],
   [169.74279832131177, -44.564510063698755],
   [169.75689798680193, -44.59369047764079],
   [169.77099771641898, -44.62287078377497],
   [16